<a href="https://www.kaggle.com/code/marianameyerf/notebook40af724143?scriptVersionId=109410399" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<a href="https://colab.research.google.com/github/jsansao/transformers_pt/blob/main/BR_BERTo_Fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inferência usando transformers pré-treinados do HuggingFace 

BR_BERTo
CENTENFolha - 8 partes
Batch size = 16
GPU T4x2

In [ ]:
#@title Passo 1:Instalando Hugging Face Transformers
# We won't need TensorFlow here
!pip uninstall -y tensorflow
# Install `transformers` from master
!pip install git+https://github.com/huggingface/transformers
!pip list | grep -E 'transformers|tokenizers'
# transformers version at notebook update --- 2.9.1
# tokenizers version at notebook update --- 0.7.0

In [ ]:
#@title Step 1: Loading the Dataset
#1.Load kant.txt using the Colab file manager
#2.Downloading the file from GitHub
#!curl -L  https://raw.githubusercontent.com/jsansao/corpus_ptbr/main/dump_Machado_Nobreak.txt --output "dump.txt"
!curl -L https://raw.githubusercontent.com/marianameyer/corpus_ptbr/main/cetenfolha_aa --output "dump.txt"
#!curl -L https://raw.githubusercontent.com/jsansao/corpus_ptbr/main/Lyrics_ChicoBuarque.txt --output "kant.txt"

!awk NF < dump.txt > kant.txt

In [ ]:
!wget https://raw.githubusercontent.com/marianameyer/corpus_ptbr/main/cetenfolha_aa 
!wget https://raw.githubusercontent.com/marianameyer/corpus_ptbr/main/cetenfolha_ab
!wget https://raw.githubusercontent.com/marianameyer/corpus_ptbr/main/cetenfolha_ac
!wget https://raw.githubusercontent.com/marianameyer/corpus_ptbr/main/cetenfolha_ad
!wget https://raw.githubusercontent.com/marianameyer/corpus_ptbr/main/cetenfolha_ae 
!wget https://raw.githubusercontent.com/marianameyer/corpus_ptbr/main/cetenfolha_af
!wget https://raw.githubusercontent.com/marianameyer/corpus_ptbr/main/cetenfolha_ag
!wget https://raw.githubusercontent.com/marianameyer/corpus_ptbr/main/cetenfolha_ah

In [ ]:
!cat cetenfolha_aa cetenfolha_ab cetenfolha_ac cetenfolha_ad cetenfolha_ae cetenfolha_af cetenfolha_ag cetenfolha_ah>  dump.txt
!awk NF < dump.txt > kant.txt

In [ ]:
#@title Passo 2:Baixando e salvando BR_BERTo
#https://huggingface.co/rdenadai/BR_BERTo

from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("rdenadai/BR_BERTo")

model = AutoModelForMaskedLM.from_pretrained("rdenadai/BR_BERTo")

In [ ]:
from datasets import load_dataset
#datasets = load_dataset("text", data_files={"train": "kant.txt", "validation": "kant_teste.txt"})
#datasets = load_dataset("text", data_files={"train": "kant.txt"}, split='train[:90%]')
ds = load_dataset("text", data_files={"train": "kant.txt"})

datasets = ds["train"].train_test_split()

In [ ]:
#@title Step 10: Building the Dataset
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="./kant.txt",
    block_size=32,
)

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [ ]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

In [ ]:
tokenized_datasets["train"][4]

In [ ]:
#block_size = tokenizer.model_max_length
block_size = 32

In [ ]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

In [ ]:
#@title Step 11: Defining a Data Collator
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [ ]:
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {metric.compute(predictions=predictions, references=labels)}

In [ ]:
#@title Step 12: Initializing the Trainer
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./KantaiBERT",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=16,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["test"],
)

In [ ]:
#@title Step 13: Pre-training the Model
trainer.train()

In [ ]:
#@title Step 14: Saving the Final Model(+tokenizer + config) to disk
#trainer.save_model("./KantaiBERT")
trainer.save_model('./BR_BERTo-CENTENFolha-8_parts')

In [ ]:
# Saving model on Wandb
import wandb
wandb.save('BR_BERTo-CENTENFolha-8_parts.h5')

In [ ]:
#@title Step 3: Configurando o pipeline fill-mask
#@title Step 15: Language Modeling with the FillMaskPipeline
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./BR_BERTo-CENTENFolha-8_parts",
    tokenizer=tokenizer
)

In [ ]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

In [ ]:
eval_results

In [ ]:
fill_mask("O rapaz olhou para o <mask> ")

In [ ]:
fill_mask("A moça olhou para o <mask> ")

In [ ]:
fill_mask("Comprou uma <mask> na loja. ")

In [ ]:
fill_mask("A mulher não é <mask>. ")

In [ ]:
fill_mask("O homem não é <mask>. ")

In [ ]:
fill_mask("Ele é um bom <mask>. ")

In [ ]:
fill_mask("Ela é uma boa <mask>. ")

In [ ]:
fill_mask("Faz de conta que ainda é <mask>. ")

In [ ]:
fill_mask("Depois da tempestade vem a <mask>. ")

In [ ]:
fill_mask("Mais vale um pássaro na mão do que <mask> voando. ")

In [ ]:
fill_mask("Não tinha <mask>, mas almoçou mesmo assim. ")

In [ ]:
fill_mask("Bebeu um copo d'água, pois tinha <mask>. ")

In [ ]:
fill_mask("Sem saber, ele descobriu um <mask>. ")

In [ ]:
fill_mask("Pedro fez fortuna vendendo <mask>. ")

In [ ]:
fill_mask("Sem medo de ser <mask>. ")